**Задание 1 Андриянов Н.А.** 

Будем решать задачу классификации и анализа новостной информации

In [1]:
# Данные загружены из https://github.com/natasha/corus#load_lenta, но через репозиторий https://github.com/yutkin
# Датасет Lenta.ru v1.0
# Распокованный размер 1.66 Gb
# Размер текст 739 351

Скачаем датасет с гитхаба

In [2]:
import requests

#url = 'https://github.com/natasha/corus/blob/master/data/lenta-ru-news.csv.gz?raw=true'
url = 'https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz'
r = requests.get(url, allow_redirects=True)
w = open('lenta-ru-news.csv.gz', 'wb').write(r.content)

Распакуем архив и занесем всю информацию в csv файл

In [3]:
import gzip
import shutil

with gzip.open('lenta-ru-news.csv.gz', 'rb') as f_in:
  with open('lenta-ru-news.csv', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

Посмотрим на данные, предварительно увеличив ширину выводимой информации в DataFrame (нам это понадобится для показа текстов статей): 

In [4]:
import pandas as pd
pd.set_option('max_colwidth',600)

df = pd.read_csv('lenta-ru-news.csv')
df.head()

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертностью от рака,"Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли...",Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины российских биатлонистов,"Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Все они продолжат выступление на Кубке мира. Полиция нагрянула в отель сборной России в Х...",Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,"Сотрудники социальной сети Instagram проанализировали поставленные пользователями смайлики, геолокации и хештеги и опубликовали итоги 2018 года. Об этом сообщается на официальном сайте Instagram. Таким образом, самой счастливой геолокацией Instagram признал Диснейленд в Токио, так как больше всего счастливых смайликов в 2018 году пользователи ставили именно под фотографиями из японского Диснейленда. Также эксперты назвали самый популярный фильтр для лица: им стал фильтр с сердечками на глазах. А, например, самыми часто используемыми хештегами в 2018 году были #metoo, #timesup и #marchforou...",Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование «российского дела»,"С начала расследования российского вмешательства в выборы власти США потратили более 25 миллионов долларов. Об этом сообщает Associated Press со ссылкой на отчет Министерства юстиции США. В документе содержатся данные о расходах на следствие с апреля по сентябрь 2018 года. За эти полгода было потрачено 4,6 миллиона долларов, из которых почти 3 миллиона долларов ушли на зарплату сотрудников, 580 тысяч — на поездки и сопутствующие расходы. Ранее Минюст США уже публиковал отчеты о затратах на дело о российском вмешательстве за предыдущие месяцы. 11 декабря расследование спецпрокурора Робера М...",Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании заминировать Севастополь,"Хакерская группировка Anonymous опубликовала новые документы о деятельности британского аналитического центра Integrity Initiative. Из материалов следует, что центр получает финансирование от некого Института государственного управления под руководством Кристофера Доннелли. Хакеры выложили в открытый доступ его паспорт, резюме, а также сведения о его связях с Министерством обороны Великобритании. Согласно документам, в 2014 году Доннелли выдвинул ряд предложений британским властям в связи с ситуацией в Крыму. В частности, он планировал заминировать Севастопольскую бухту, окружить полуостро...",Мир,Общество


Проверим таблицу на наличие пустых данных

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739351 entries, 0 to 739350
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     739351 non-null  object
 1   title   739351 non-null  object
 2   text    739346 non-null  object
 3   topic   739148 non-null  object
 4   tags    712837 non-null  object
dtypes: object(5)
memory usage: 28.2+ MB


Не все поля таблицы заполнены, но с учетом размера данных выберем такие, где значение параметра tags, text и topic не пустое

In [6]:
#df_full = df[(df['text'].isnull()==False)&(df['topic'].isnull()==False)&(df['tags'].isnull()==False)]
df_full = df.dropna()

In [7]:
df_full

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертностью от рака,"Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли...",Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины российских биатлонистов,"Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Все они продолжат выступление на Кубке мира. Полиция нагрянула в отель сборной России в Х...",Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,"Сотрудники социальной сети Instagram проанализировали поставленные пользователями смайлики, геолокации и хештеги и опубликовали итоги 2018 года. Об этом сообщается на официальном сайте Instagram. Таким образом, самой счастливой геолокацией Instagram признал Диснейленд в Токио, так как больше всего счастливых смайликов в 2018 году пользователи ставили именно под фотографиями из японского Диснейленда. Также эксперты назвали самый популярный фильтр для лица: им стал фильтр с сердечками на глазах. А, например, самыми часто используемыми хештегами в 2018 году были #metoo, #timesup и #marchforou...",Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование «российского дела»,"С начала расследования российского вмешательства в выборы власти США потратили более 25 миллионов долларов. Об этом сообщает Associated Press со ссылкой на отчет Министерства юстиции США. В документе содержатся данные о расходах на следствие с апреля по сентябрь 2018 года. За эти полгода было потрачено 4,6 миллиона долларов, из которых почти 3 миллиона долларов ушли на зарплату сотрудников, 580 тысяч — на поездки и сопутствующие расходы. Ранее Минюст США уже публиковал отчеты о затратах на дело о российском вмешательстве за предыдущие месяцы. 11 декабря расследование спецпрокурора Робера М...",Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании заминировать Севастополь,"Хакерская группировка Anonymous опубликовала новые документы о деятельности британского аналитического центра Integrity Initiative. Из материалов следует, что центр получает финансирование от некого Института государственного управления под руководством Кристофера Доннелли. Хакеры выложили в открытый доступ его паспорт, резюме, а также сведения о его связях с Министерством обороны Великобритании. Согласно документам, в 2014 году Доннелли выдвинул ряд предложений британским властям в связи с ситуацией в Крыму. В частности, он планировал заминировать Севастопольскую бухту, окружить полуостро...",Мир,Общество
...,...,...,...,...,...
739346,https://lenta.ru/news/1999/08/31/cholera/,Южно-Сахалинск объявлен очагом холеры,"Сегодня областной центр Сахалина и Курил получил статус очага распространения холеры. Как сообщает ИТАР-ТАСС со ссылкой на пресс-центр администрации Сахалинской области, в лечебных учреждениях Южно-Сахалинска уже находятсятся 5 горожан, причем у двоих

Убедимся, что данные без пропусков

In [8]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712829 entries, 0 to 739350
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     712829 non-null  object
 1   title   712829 non-null  object
 2   text    712829 non-null  object
 3   topic   712829 non-null  object
 4   tags    712829 non-null  object
dtypes: object(5)
memory usage: 32.6+ MB


Видим, что помимо полных текстовых описаний в таблице присутствуют столбцы topic и tags. Для классификации выберем тот, в котором меньше уникальных значений.

In [9]:
topics = set(df_full['topic'])
tags = set(df_full['tags'])
print('Topics:', topics, len(topics))
print('Tags:', tags, len(tags))

Topics: {'Бизнес', 'Оружие', 'Наука и техника', 'Дом', 'Культура', 'Силовые структуры', 'Библиотека', 'Легпром', 'Интернет и СМИ', 'ЧМ-2014', 'Путешествия', 'Ценности', 'Бывший СССР', 'Мир', 'Культпросвет ', 'Сочи', 'Крым', 'Из жизни', 'Спорт', 'МедНовости', 'Россия', 'Экономика', '69-я параллель'} 23
Tags: {'Явления', 'Преступность', 'Полиция и спецслужбы', 'Офис', 'Молдавия', 'Реклама', 'Гаджеты', 'Политика', 'Вооружение', 'Театр', 'Coцсети', 'Зимние виды', 'Софт', 'Инструменты', 'Мнения', 'Вирусные ролики', 'Первая мировая', 'Госэкономика', 'Футбол', 'Движение', 'Общество', 'Финансы компаний', 'Бизнес', 'Оружие', 'Жизнь', 'Звери', 'Выборы', 'Москва', 'Белоруссия', 'Страноведение', 'События', 'Автобизнес', 'Мировой бизнес', 'Люди', 'Достижения', 'История', 'Вещи', 'Происшествия', 'Киберпреступность', 'Хоккей', 'Квартира', 'Конфликты', 'Следствие и суд', 'Мир', 'Мировой опыт', 'Прибалтика', 'Рынки', 'Вкусы', 'Пресса', 'Закавказье', 'Туризм', 'Кино', 'Госрегулирование', 'Фотография', '

Таким образом, классификация возможна по 9 темам. Тэгов слишком много - 88. В итоговый датасет попало более, чем 712 тысяч строк, поэтому сократим его в 20 раз с целью оптимизации процессов анализа и обучения модели.

In [10]:
df_cut = df_full.sample(frac=0.005, random_state=1) # датасет большой, поэтому берем треть, оставим всегда одинаковые результаты
df_cut.reset_index(inplace=True, drop=True)
df_text = df_cut['text'].tolist()
df_topic = df_cut['topic'].tolist()
topic_cnt = len(set(df_topic))
print(f'Количество тем для классификации после обрезки: {topic_cnt}')
df_cut.head()

Количество тем для классификации после обрезки: 18


,url,title,text,topic,tags
0,https://lenta.ru/news/2011/02/04/developing/,Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю,"Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в четвертом квартале (рост на 2,9 процента в годовом исчислении), а также...",Экономика,Все
1,https://lenta.ru/news/2016/08/12/funeral/,Брат и сестра погибли в Кировской области по пути на похороны матери,"В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в августе в Астрахани автобус, водитель которого потерял сознание во время е...",Россия,Происшествия
2,https://lenta.ru/news/2006/11/21/kaladze/,"Защитник ""Милана"" будет рекламировать грузинское вино","Защитник сборной Грузии и итальянского ""Милана"" Каха Каладзе решил рекламировать и продавать грузинские вина в знаменитом миланском ресторане ""Джанино"". В июне футболист вместе со своими друзьями выкупил и отреставрировал это заведение и теперь собирается добавить в меню свои национальные вина. ""В нашем ресторане бывают известные лица Италии: спортсмены, бизнесмены, журналисты, актеры, модели. Пока мы продаем итальянские вина. Скоро в ресторане появятся грузинские вина, мы будем их рекламировать и продавать"", - рассказал Каладзе. Как отмечает интернет-издание korrespondent.net, пока неизв...",Спорт,Все
3,https://lenta.ru/news/2004/05/14/artifacts/,Шведский археолог просит снять с аукционов ножи викингов,"Археолог Кристиан Рюнебю (Christian Runeby) просит правительство Швеции вернуть в страну оружие викингов, которое оказалось в распоряжении аукциона Faganarms в штате Мичиган, сообщает АР. Рюнебю обнаружил на сайте этого аукциона, который специализируется на старинном оружии, выставленные мечи, ножи, браслеты и конскую сбрую. Большинство артефактов датируется примерно 800 годом нашей эры. Общая стоимость коллекции из сотни предметов оценивается администрацией Faganarms в 26 тысяч долларов. На сайте указано, что все это было найдено на раскопках одного поселения викингов на острове Готланд...",Мир,Все
4,https://lenta.ru/news/2005/05/23/elections/,Аббаса просят отложить парламентские выборы,"В понедельник палестинский избирком объявил, что не успеет подготовиться к проведению парламентских выборов, намеченных на 17 июля. Как сообщает агентство Associated Press, в связи с этим председателя палестинской автономии Махмуда Аббаса попросили назначить специальным указом новую дату выборов. Комиссия утверждает, что ей потребуется не менее двух месяцев с момента принятия закона о выборах, чтобы подготовиться к их проведению. Однако принятие выборного законодательства откладывается из-за разногласий между Аббасом и парламентом, которые никак не могут договориться, сколько депутатов дол...",Мир,Все


Оставим только поля title, text, topic. Но добавим поле full description, которое будет содержать и текст и заголовок статьи. Потом можно попробовать реализовать 3 модели.

In [11]:
df_cut = df_cut[['title', 'text', 'topic']]
df_cut.insert(2, 'full_description', pd.Series([i for i in range(len(df_cut))]))
df_cut['full_description'] = df_cut['title'] + "." + df_cut['text']
df_text = df_cut['text'].tolist()
df_title = df_cut['title'].tolist()
df_fulltext = df_cut['full_description'].to_list()
df_cut.head()

,title,text,full_description,topic
0,Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю,"Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в четвертом квартале (рост на 2,9 процента в годовом исчислении), а также...","Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю.Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в...",Экономика
1,Брат и сестра погибли в Кировской области по пути на похороны матери,"В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в августе в Астрахани автобус, водитель которого потерял сознание во время е...","Брат и сестра погибли в Кировской области по пути на похороны матери.В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в авгус...",Россия
2,"Защитник ""Милана"" будет рекламировать грузинское вино","Защитник сборной Грузии и итальянского ""Милана"" Каха Каладзе решил рекламировать и продавать грузинские вина в знаменитом миланском ресторане ""Джанино"". В июне футболист вместе со своими друзьями выкупил и отреставрировал это заведение и теперь собирается добавить в меню свои национальные вина. ""В нашем ресторане бывают известные лица Италии: спортсмены, бизнесмены, журналисты, актеры, модели. Пока мы продаем итальянские вина. Скоро в ресторане появятся грузинские вина, мы будем их рекламировать и продавать"", - рассказал Каладзе. Как отмечает интернет-издание korrespondent.net, пока неизв...","Защитник ""Милана"" будет рекламировать грузинское вино.Защитник сборной Грузии и итальянского ""Милана"" Каха Каладзе решил рекламировать и продавать грузинские вина в знаменитом миланском ресторане ""Джанино"". В июне футболист вместе со своими друзьями выкупил и отреставрировал это заведение и теперь собирается добавить в меню свои национальные вина. ""В нашем ресторане бывают известные лица Италии: спортсмены, бизнесмены, журналисты, актеры, модели. Пока мы продаем итальянские вина. Скоро в ресторане появятся грузинские вина, мы будем их рекламировать и продавать"", - рассказал Каладзе. Как от...",Спорт
3,Шведский археолог просит снять с аукционов ножи викингов,"Археолог Кристиан Рюнебю (Christian Runeby) просит пра

Проверим, что тексты чистые. Выведем несколько примеров из начала, середины и конца таблицы

In [12]:
for t in df_cut['text'][:10]:
  print(t)
for t in df_cut['text'][25000:25010]:
  print(t)
for t in df_cut['text'][-10:]:
  print(t)

Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в четвертом квартале (рост на 2,9 процента в годовом исчислении), а также по промышленному производству в январе. Наибольший урон понесли фонды Китая и Индии, отток средств из которых превысил 800 миллионов долларов. Напротив, Восточная Европа и России столкнулись с проблемой бегства капиталов в значительно меньшей степени. Как отмечают "Ведомости", сравнительно неплохие показатели России могут быть обусловлены интересом инвесторов к РФ как к альтернативному поставщику эне

<b>Предобработка текста</b>

Естественно, просто так начать работу с текстами не выйдет, поэтому необходимо:
1. Токенизировать тексты
2. Убрать стоп-слова и пунктуацию
3. Лемматизировать токены

In [13]:
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=d12f790551ab378e49ac6f0df6dd9da0de5098c10c32e65ddde39fba8baaac6a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from functools import lru_cache
from tqdm import tqdm  
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize # выполняется очень долго, лучше использовать regex
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer(lang='ru') # для лемматизации
stop_words = stopwords.words('russian') # стоп слова
stop_words.extend(['быть', 'что', 'это', 'так', 'вот', 'как', 'к', 'в', 'на', '...', '—', '–', '-']) #дополним список стоп-слов

def word_tokenize(text): # оптимизируем, используя regex вместо nltk.tokenize
  text = text.lower() # весь текст строчными буквами
  text = re.sub(r'[^a-zA-Zа-яА-Я —–-]', '', str(text)) # выделяем только слова, в том числе с тире
  words = text.split() # делим на слова весь фильтрованный текст
  return words

@lru_cache(maxsize=None) # оптимизируем, неограничено сохраняя в кэш значения функции
def word_lemmatize(word):
  word = morph.parse(word)[0].normal_form # начальная форма с лучшим совпадением (score)
  return word

def process_text(text):
  text_result = () # оптимизируем, используя кортеж
  for article in tqdm(text):
    tokens = word_tokenize(article) # токенизируем тексты
    tokens = filter(lambda word: word not in stop_words, tokens) # фильтруем стопслова
    lemmas = tuple(word_lemmatize(word) for word in tokens) # лемматизируем
    text_result += (lemmas,) # оптимизируем, используя кортеж
  return text_result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
text_lemmatized = process_text(df_text) # с помощью оптимизации обработка сократилась

100%|██████████| 3564/3564 [00:34<00:00, 104.58it/s]


In [16]:
title_lemmatized = process_text(df_title)

100%|██████████| 3564/3564 [00:01<00:00, 2645.55it/s]


In [17]:
fulltext_lemmatized = process_text(df_fulltext)

100%|██████████| 3564/3564 [00:04<00:00, 786.59it/s]


Выведем некоторые примеры лемматизации

In [18]:
text_lemmatized[:10]

(('инвестор',
  'неделя',
  'январь',
  'февраль',
  'резко',
  'сократить',
  'свой',
  'вложение',
  'акция',
  'развивающийся',
  'рынок',
  'сообщать',
  'reuters',
  'чистый',
  'отток',
  'капитал',
  'соответствующий',
  'фонд',
  'составить',
  'миллиард',
  'доллар',
  'один',
  'ключевой',
  'причина',
  'вывод',
  'средство',
  'стать',
  'беспорядок',
  'египет',
  'грозить',
  'усиление',
  'политический',
  'нестабильность',
  'развивающийся',
  'страна',
  'значительный',
  'часть',
  'этот',
  'средство',
  'перевести',
  'ценный',
  'бумага',
  'сша',
  'что',
  'способствовать',
  'хороший',
  'макроэкономический',
  'статистика',
  'частность',
  'выйти',
  'убедительный',
  'дать',
  'рост',
  'ввп',
  'четвёртый',
  'квартал',
  'рост',
  'процент',
  'годовой',
  'исчисление',
  'также',
  'промышленный',
  'производство',
  'январь',
  'больший',
  'урон',
  'понести',
  'фонд',
  'китай',
  'индия',
  'отток',
  'средство',
  'который',
  'превысить',
  'миллион

## Выделение тем из текстов

Построим LDA тематическую модель, оценим её работоспособность, выведя характерные слова для тем. 
Так как количество тем можно подбирать в первую очередь "на глаз", адекватность работы модели предлагается осуществить самостоятельно.

In [19]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # игорирование предупреждения модуля gensim
warnings.filterwarnings("ignore", category=FutureWarning) # игорирование предупреждения модуля pyLDAvis

In [20]:
import gensim.corpora as corpora
def make_corpus(text):
  # Словарь (id2word) - уникальный идентификатор для каждого слова word, word_id
  id2word = corpora.Dictionary(text)
  # Корпус (corpus) - количествл повторений для каждого слова word_id, word_frequency
  corpus = [id2word.doc2bow(t) for t in text]
  print(*tuple(tuple({'ID': id, 'Слово': id2word[id], 'Повторений': freq} for id, freq in cp) for cp in corpus[:1])[0], sep='\n')
  return id2word, corpus

Рассмотрим на примере лемматизированного текста

In [21]:
id2word_text, corpus_text = make_corpus(text_lemmatized)

{'ID': 0, 'Слово': 'reuters', 'Повторений': 1}
{'ID': 1, 'Слово': 'акция', 'Повторений': 1}
{'ID': 2, 'Слово': 'альтернативный', 'Повторений': 1}
{'ID': 3, 'Слово': 'бегство', 'Повторений': 1}
{'ID': 4, 'Слово': 'беспорядок', 'Повторений': 1}
{'ID': 5, 'Слово': 'ближний', 'Повторений': 1}
{'ID': 6, 'Слово': 'больший', 'Повторений': 1}
{'ID': 7, 'Слово': 'бумага', 'Повторений': 1}
{'ID': 8, 'Слово': 'ввп', 'Повторений': 1}
{'ID': 9, 'Слово': 'ведомость', 'Повторений': 1}
{'ID': 10, 'Слово': 'весь', 'Повторений': 1}
{'ID': 11, 'Слово': 'вложение', 'Повторений': 1}
{'ID': 12, 'Слово': 'восток', 'Повторений': 1}
{'ID': 13, 'Слово': 'восточный', 'Повторений': 1}
{'ID': 14, 'Слово': 'вывод', 'Повторений': 1}
{'ID': 15, 'Слово': 'выйти', 'Повторений': 1}
{'ID': 16, 'Слово': 'годовой', 'Повторений': 1}
{'ID': 17, 'Слово': 'грозить', 'Повторений': 1}
{'ID': 18, 'Слово': 'дать', 'Повторений': 1}
{'ID': 19, 'Слово': 'доллар', 'Повторений': 2}
{'ID': 20, 'Слово': 'европа', 'Повторений': 1}
{'ID': 

In [22]:
id2word_title, corpus_title = make_corpus(title_lemmatized)

{'ID': 0, 'Слово': 'доллар', 'Повторений': 1}
{'ID': 1, 'Слово': 'занеделя', 'Повторений': 1}
{'ID': 2, 'Слово': 'инвестиция', 'Повторений': 1}
{'ID': 3, 'Слово': 'миллиард', 'Повторений': 1}
{'ID': 4, 'Слово': 'потерять', 'Повторений': 1}
{'ID': 5, 'Слово': 'развивающийся', 'Повторений': 1}
{'ID': 6, 'Слово': 'страна', 'Повторений': 1}


In [23]:
id2word_fulltext, corpus_fulltext = make_corpus(fulltext_lemmatized)

{'ID': 0, 'Слово': 'reuters', 'Повторений': 1}
{'ID': 1, 'Слово': 'акция', 'Повторений': 1}
{'ID': 2, 'Слово': 'альтернативный', 'Повторений': 1}
{'ID': 3, 'Слово': 'бегство', 'Повторений': 1}
{'ID': 4, 'Слово': 'беспорядок', 'Повторений': 1}
{'ID': 5, 'Слово': 'ближний', 'Повторений': 1}
{'ID': 6, 'Слово': 'больший', 'Повторений': 1}
{'ID': 7, 'Слово': 'бумага', 'Повторений': 1}
{'ID': 8, 'Слово': 'ввп', 'Повторений': 1}
{'ID': 9, 'Слово': 'ведомость', 'Повторений': 1}
{'ID': 10, 'Слово': 'весь', 'Повторений': 1}
{'ID': 11, 'Слово': 'вложение', 'Повторений': 1}
{'ID': 12, 'Слово': 'восток', 'Повторений': 1}
{'ID': 13, 'Слово': 'восточный', 'Повторений': 1}
{'ID': 14, 'Слово': 'вывод', 'Повторений': 1}
{'ID': 15, 'Слово': 'выйти', 'Повторений': 1}
{'ID': 16, 'Слово': 'годовой', 'Повторений': 1}
{'ID': 17, 'Слово': 'грозить', 'Повторений': 1}
{'ID': 18, 'Слово': 'дать', 'Повторений': 1}
{'ID': 19, 'Слово': 'доллар', 'Повторений': 3}
{'ID': 20, 'Слово': 'европа', 'Повторений': 1}
{'ID': 

In [24]:
# Для обучения модели LDA нужно указать корпус (corpus), словарь (id2word) и количество тем (num_topics)
# Максимальное количество итераций по корпусу при выводе тематического распределения корпуса (iterations)

In [25]:
import gensim
def make_model(my_corpus, my_id2word, num_iter=5):
  # Модель LDA
  lda_model = gensim.models.ldamodel.LdaModel(corpus=my_corpus, id2word=my_id2word,
                                            num_topics=topic_cnt, iterations=num_iter) # итерация занимает, как правило, около 30 секунд
  return lda_model

In [26]:
lda_model_text = make_model(corpus_text,id2word_text)

In [27]:
lda_model_title = make_model(corpus_title,id2word_title)

In [28]:
lda_model_fulltext = make_model(corpus_fulltext,id2word_fulltext)

In [29]:
print(lda_model_text)

LdaModel<num_terms=38194, num_topics=18, decay=0.5, chunksize=2000>


## Анализ получившихся тем

По результатам предыдущего этапа у нас получилась приемлемая тематическая модель. Теперь нужно качественно определить, действительно ли она хороша:
* Все ли темы из датасета освещены в темах?
* Насколько сильно темы пересекаются друг с другом?
* Для каждой темы определим её название (1-2 слова)

In [30]:
from gensim.models import CoherenceModel

# Определяем качество модели
def evaluate_lda(lda_model, corpus, text, id2word):
  lda_perplexity = lda_model.log_perplexity(corpus) # сложность модели
  lda_coherence = CoherenceModel(model=lda_model, texts=text_lemmatized, dictionary=id2word, coherence='c_v') # согласованность тем
  lda_coherence = lda_coherence.get_coherence() # аккуратно, выполнение долгое
  print('Cложность модели:', lda_perplexity) # чем ниже, тем модель лучше
  print('Согласованность тем:', lda_coherence) # чем выше, тем модель лучше

In [31]:
evaluate_lda(lda_model_text, corpus_text, text_lemmatized, id2word_text)

Cложность модели: -10.864782141626998
Согласованность тем: 0.2630660988899201


In [32]:
evaluate_lda(lda_model_title, corpus_title, title_lemmatized, id2word_title)

/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


Cложность модели: -13.018929928865296
Согласованность тем: nan


В модели по заголовкам мало информации

In [33]:
evaluate_lda(lda_model_fulltext, corpus_fulltext, fulltext_lemmatized, id2word_fulltext)

Cложность модели: -11.04148802250951
Согласованность тем: 0.2629244041749769


Полная модель лишь незначительно превосходит модель на основе текстов статей без заголовков

Визуализируем результаты тематической модели

In [34]:
#!pip install pyLDAvis

In [35]:
!pip install pyLDAvis==2.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97736 sha256=1d8d4cce05657984786cb5585bb237ea148ec817af62750edd9a338115f4e5b7
  Stored in directory: /root/.cache/pip/wheels/d9/93/d6/16c95da19c32f037fd75135ea152d0df37254c25cd1a8b4b6c
Successfully built pyLDAvis


In [38]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis
pyLDAvis.enable_notebook()

# Визуализируем и посмотрим насколько сильно темы пересекаются друг с другом
lda_viz = gensimvis.prepare(lda_model_text, corpus_text, id2word_text)
lda_viz

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
17     0.007129  0.005968       1        1  6.804716
13    -0.000453  0.005541       2        1  6.316936
6     -0.000379  0.001814       3        1  6.075507
10    -0.007324  0.004608       4        1  6.054549
2     -0.000538 -0.002184       5        1  5.873074
16     0.000676  0.002765       6        1  5.798627
3      0.000925 -0.003044       7        1  5.622994
14     0.002053  0.002118       8        1  5.610627
8     -0.003174 -0.001574       9        1  5.567634
7     -0.004011 -0.001549      10        1  5.542402
0      0.000511 -0.000013      11        1  5.531284
15     0.001462  0.000992      12        1  5.379638
5     -0.003613 -0.001390      13        1  5.243814
1     -0.002006 -0.003012      14        1  5.137684
4     -0.002440 -0.001151      15        1  5.115332
11     0.003644 -0.007082      16        1  4.928786
12     0.001518 -0.001333      17        1  4.880933
9      0.006019 -0.001474      18        1  4.515463, topic_info=           Term         Freq        Total Category  logprob  loglift
165         год  5834.000000  5834.000000  Default  30.0000  30.0000
32      который  4285.000000  4285.000000  Default  29.0000  29.0000
57      процент  1483.000000  1483.000000  Default  28.0000  28.0000
67     сообщать  2058.000000  2058.000000  Default  27.0000  27.0000
839   президент  1232.000000  1232.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
37      миллион    42.505539  1047.956723  Topic18  -6.1966  -0.1073
839   президент    43.860518  1232.511039  Topic18  -6.1652  -0.2381
57      процент    45.523838  1483.087940  Topic18  -6.1280  -0.3860
844  российский    42.865582  1379.887724  Topic18  -6.1882  -0.3740
830      первый    40.044923  1155.687199  Topic18  -6.2563  -0.2648

[1395 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
96        1  0.076034   -летний
96        2  0.039044   -летний
96        3  0.055484   -летний
96        4  0.059594   -летний
96        5  0.063704   -летний
...     ...       ...       ...
230      17  0.059582  являться
230      18  0.033677  являться
8813      6  0.319703       янг
8813      9  0.159852       янг
8813     12  0.159852       янг

[4713 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 14, 7, 11, 3, 17, 4, 15, 9, 8, 1, 16, 6, 2, 5, 12, 13, 10])

In [39]:
# Полнота освещения тем
# Количество тем в датасете 23
# Количество тем в модели 19

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# Ключевые слова для каждой темы с весом (важностью)
print(*sorted(lda_model_text.print_topics(), key=lambda x: x[0]), sep='\n')

(0, '0.015*"год" + 0.010*"который" + 0.005*"россия" + 0.005*"время" + 0.004*"слово" + 0.004*"сообщать" + 0.004*"человек" + 0.004*"заявить" + 0.004*"свой" + 0.003*"также"')
(1, '0.013*"год" + 0.007*"который" + 0.005*"также" + 0.005*"президент" + 0.004*"российский" + 0.004*"россия" + 0.004*"процент" + 0.004*"сообщать" + 0.003*"компания" + 0.003*"тысяча"')
(2, '0.011*"год" + 0.010*"который" + 0.006*"сообщать" + 0.005*"время" + 0.004*"компания" + 0.004*"также" + 0.004*"человек" + 0.004*"россия" + 0.004*"российский" + 0.003*"первый"')
(3, '0.012*"год" + 0.010*"который" + 0.007*"россия" + 0.005*"сообщать" + 0.004*"человек" + 0.004*"процент" + 0.003*"страна" + 0.003*"заявить" + 0.003*"один" + 0.003*"российский"')
(4, '0.012*"год" + 0.009*"который" + 0.006*"россия" + 0.005*"также" + 0.005*"сообщать" + 0.004*"компания" + 0.004*"свой" + 0.003*"президент" + 0.003*"человек" + 0.003*"процент"')
(5, '0.015*"год" + 0.008*"который" + 0.006*"сообщать" + 0.005*"также" + 0.005*"россия" + 0.004*"время" + 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
# Это означает, что тему 2 можно описать словами: политика
# При этом у слова "президент" вес (важность) один, а у слова "россия" - другой и т. д.

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
# Возможно для улучшения тематической модели стоит убрать английсие слова
# и в стоп слова добавить: который, также, такой, твой, свой

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Модель классификации**

In [43]:
df_cut.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,title,text,full_description,topic
0,Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю,"Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в четвертом квартале (рост на 2,9 процента в годовом исчислении), а также...","Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю.Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в...",Экономика
1,Брат и сестра погибли в Кировской области по пути на похороны матери,"В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в августе в Астрахани автобус, водитель которого потерял сознание во время е...","Брат и сестра погибли в Кировской области по пути на похороны матери.В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в авгус...",Россия
2,"Защитник ""Милана"" будет рекламировать грузинское вино","Защитник сборной Грузии и итальянского ""Милана"" Каха Каладзе решил рекламировать и продавать грузинские вина в знаменитом миланском ресторане ""Джанино"". В июне футболист вместе со своими друзьями выкупил и отреставрировал это заведение и теперь собирается добавить в меню свои национальные вина. ""В нашем ресторане бывают известные лица Италии: спортсмены, бизнесмены, журналисты, актеры, модели. Пока мы продаем итальянские вина. Скоро в ресторане появятся грузинские вина, мы будем их рекламировать и продавать"", - рассказал Каладзе. Как отмечает интернет-издание korrespondent.net, пока неизв...","Защитник ""Милана"" будет рекламировать грузинское вино.Защитник сборной Грузии и итальянского ""Милана"" Каха Каладзе решил рекламировать и продавать грузинские вина в знаменитом миланском ресторане ""Джанино"". В июне футболист вместе со своими друзьями выкупил и отреставрировал это заведение и теперь собирается добавить в меню свои национальные вина. ""В нашем ресторане бывают известные лица Италии: спортсмены, бизнесмены, журналисты, актеры, модели. Пока мы продаем итальянские вина. Скоро в ресторане появятся грузинские вина, мы будем их рекламировать и продавать"", - рассказал Каладзе. Как от...",Спорт
3,Шведский археолог просит снять с аукционов ножи викингов,"Археолог Кристиан Рюнебю (Christian Runeby) просит пра

Сделаем функцию преобразования текстовой информации

In [44]:
from nltk.stem import SnowballStemmer
def news_to_words(news_text):
    news_text = news_text.replace('ё','е')
    news_text = news_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", news_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stop_words]
    # 4. Делаем стемминг
    snowball = SnowballStemmer(language='russian')
    stem_words = [snowball.stem(w) for w in meaningful_words]
    # 5. Формируем текст, объединяя слова через пробел
    return( " ".join(stem_words ))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Закодируем классы модели

In [45]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df_cut['topic'])
df_cut['topic'] = le.transform(df_cut['topic'])
df_cut[:10]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,title,text,full_description,topic
0,Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю,"Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в четвертом квартале (рост на 2,9 процента в годовом исчислении), а также...","Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю.Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в...",17
1,Брат и сестра погибли в Кировской области по пути на похороны матери,"В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в августе в Астрахани автобус, водитель которого потерял сознание во время е...","Брат и сестра погибли в Кировской области по пути на похороны матери.В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в авгус...",13
2,"Защитник ""Милана"" будет рекламировать грузинское вино","Защитник сборной Грузии и итальянского ""Милана"" Каха Каладзе решил рекламировать и продавать грузинские вина в знаменитом миланском ресторане ""Джанино"". В июне футболист вместе со своими друзьями выкупил и отреставрировал это заведение и теперь собирается добавить в меню свои национальные вина. ""В нашем ресторане бывают известные лица Италии: спортсмены, бизнесмены, журналисты, актеры, модели. Пока мы продаем итальянские вина. Скоро в ресторане появятся грузинские вина, мы будем их рекламировать и продавать"", - рассказал Каладзе. Как отмечает интернет-издание korrespondent.net, пока неизв...","Защитник ""Милана"" будет рекламировать грузинское вино.Защитник сборной Грузии и итальянского ""Милана"" Каха Каладзе решил рекламировать и продавать грузинские вина в знаменитом миланском ресторане ""Джанино"". В июне футболист вместе со своими друзьями выкупил и отреставрировал это заведение и теперь собирается добавить в меню свои национальные вина. ""В нашем ресторане бывают известные лица Италии: спортсмены, бизнесмены, журналисты, актеры, модели. Пока мы продаем итальянские вина. Скоро в ресторане появятся грузинские вина, мы будем их рекламировать и продавать"", - рассказал Каладзе. Как от...",15
3,Шведский археолог просит снять с аукционов ножи викингов,"Археолог Кристиан Рюнебю (Christian Runeby) просит правительство Шве

Применим функцию к нашему обрезанному датасету

In [46]:
bu = df_cut.copy
df_cut['txt'] = df_cut['text'].apply(lambda s: news_to_words(s))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
df_cut.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,title,text,full_description,topic,txt
0,Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю,"Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в четвертом квартале (рост на 2,9 процента в годовом исчислении), а также...","Развивающиеся страны потеряли 4 миллиарда долларов инвестиций за неделю.Инвесторы на неделе с 26 января по 2 февраля резко сократили свои вложения в акции развивающихся рынков, сообщает Reuters. Чистый отток капитала из соответствующих фондов составил 4,1 миллиарда долларов. Одной из ключевых причин вывода средств стали беспорядки в Египте, грозящие усилением политической нестабильности в развивающихся странах. Значительная часть этих средств была переведена в ценные бумаги США, чему способствовала хорошая макроэкономическая статистика. В частности, вышли убедительные данные по росту ВВП в...",17,инвестор недел январ феврал резк сократ сво вложен акц развива рынк сообща чист отток капита соответств фонд состав миллиард доллар одн ключев причин вывод средств стал беспорядк египт грозя усилен политическ нестабильн развива стран значительн част эт средств перевед цен бумаг сша чем способствова хорош макроэкономическ статистик частност вышл убедительн дан рост ввп четверт квартал рост процент годов исчислен такж промышлен производств январ наибольш урон понесл фонд кит инд отток средств котор превыс миллион доллар напрот восточн европ росс столкнул проблем бегств капитал значительн мен...
1,Брат и сестра погибли в Кировской области по пути на похороны матери,"В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в августе в Астрахани автобус, водитель которого потерял сознание во время е...","Брат и сестра погибли в Кировской области по пути на похороны матери.В Кировской области брат и сестра, ехавшие на похороны матери, погибли в ДТП, сообщает в пятницу, 12 августа, портал Progorod43.ru. Авария произошла накануне утром на трассе в Омутнинском районе области. Находившийся за рулем автомобиля ВАЗ 69-летний мужчина не справился с управлением, из-за чего машина съехала в кювет и перевернулась. В результате ДТП погибли водитель и 59-летняя пассажирка. Отмечается, что они ехали на церемонию прощания с 91-летней матерью, проходившую в деревне Челка в Советском районе. Ранее в авгус...",13,кировск област брат сестр еха похорон матер погибл дтп сообща пятниц август порта авар произошл наканун утр трасс омутнинск район област наход рул автомобил ваз летн мужчин справ управлен машин съеха кювет перевернул результат дтп погибл водител летн пассажирк отмеча еха церемон прощан летн матер проход деревн челк советск район ран август астрахан автобус водител котор потеря сознан врем езд протаран нескольк автомобил сбил двух пешеход пострада госпитализирова различн травм наход рул мужчин сконча
2,"Защитник ""Милана"" будет рекламировать грузинское вино","Защитник сборной Грузии и итальянского ""Милана"" Каха Каладзе решил рекламировать и продавать грузинские вина в знаменитом миланском ресторане ""Джанино"". В июне футболист вместе со своими друзьями выкупил и отреставрировал это заведени

Построим мешок слов

In [48]:
from sklearn import feature_extraction
from sklearn import model_selection

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
bow = feature_extraction.text.CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
bagOfWords = bow.fit_transform(df_cut['txt'])
#vocabulary=bow.get_feature_names()
vocabulary=bow.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
print(vocabulary)

['ааг' 'аар' 'ааронович' ... 'ящер' 'ящериц' 'ящик']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Поделим на обучающую и тестовую выборку

In [52]:
bagOfWordsTrain, bagOfWordsTest, yTrain, yTest = model_selection.train_test_split(bagOfWords, df_cut['topic'], test_size=0.2, random_state=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Создадим машину опорных векторов svm.LinearSVC с параметрами C=1, fit_intercept=False.

Натренируем ее на обучающей выборке и с помощью метода .score(x,y) вычислим процент правильных предсказаний для тестовой выборки.

In [53]:
from sklearn import svm
SVM = svm.LinearSVC(C=0.1, fit_intercept=False)
SVM.fit(bagOfWordsTrain, yTrain)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LinearSVC(C=0.1, fit_intercept=False)

In [54]:
SVM.score(bagOfWordsTest,yTest)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.7237026647966339

У модели довольно высокая точность на тестовой выборке. Проанализируем произвольный текст.

In [92]:
my_txt = ['Роджер Федерер обыграл Рафаэля Надаля в финале теннисного турнира в Австралии. Счет матча 6:2, 6:0. Федерер\
играл намного лучше, брал гейм за геймом. Он поднял кубок над головой, проведя потрясающий матч.', 'Посольство России готово к любым \
решениям Москвы после захвата польскими властями здания российской школы. Об этом заявил ТАСС посол РФ в Польше Сергей Андреев, \
отвечая на вопрос о возможности разрыва дипломатических отношений.']
txt_transform = bow.transform(my_txt)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [93]:
class_id = SVM.predict(txt_transform)
print(class_id)

[15 13]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
labels = list(le.inverse_transform(class_id))
print(labels)

['Спорт', 'Россия']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Модель распознала предложенные тексты, как "Спорт" и "Россия", что может быть адекватно.

In [ ]:
#можно использовать больше данных
#построить более сложную модель, но время поджимает